In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree,ensemble,metrics


In [3]:
df = pd.read_csv('dataset_results.csv')


In [4]:
df = df[df.results.notna()]

In [5]:
df = df.drop(df[df.results == 'Time limit reached.\r\n'].index)
df = df.drop(df[df.results == 'No solution exists.\r\n'].index)
#df = df.drop(df[df.results == 'key_action1(char_acting)\r\n'].index)
#df = df.drop(df[df.results == 'key_action2(char_acting)\r\n'].index)
#df = df.drop(df[df.results == 'key_action3(char_acting)\r\n'].index)
#df = df.drop(df[df.results == 'unblock_yourself(char_acting)'].index)
df.reset_index(inplace=True,drop=True)


In [6]:
df.head(5)

,scenario,acting_char,Alice_alive,Alice_underArrest,Alice_angry,Alice_suspect,Alice_at,Bob_alive,Bob_underArrest,Bob_angry,...,Theft_Bat_BasketballCourt_clues,Theft_Bat_Downtown_clues,Murder_Basketball_HomeB_clues,Murder_Basketball_BasketballCourt_clues,Murder_Basketball_Downtown_clues,Murder_Bat_HomeB_clues,Murder_Bat_BasketballCourt_clues,Murder_Bat_Downtown_clues,results,time
0,1,Alice,1,0,1,2,HomeB,1,0,1,...,0,0,0,0,0,0,0,0,"travel(Alice, HomeB, Downtown) steal(Alice, Bo...",0.708981
1,1,Charlie,1,0,1,2,BasketballCourt,1,0,0,...,0,0,0,0,0,0,0,0,"travel(Charlie, Downtown, BasketballCourt) kil...",0.701070
2,1,Bob,1,0,1,2,BasketballCourt,1,0,0,...,0,0,0,0,0,0,0,0,"travel(Bob, HomeB, BasketballCourt) steal(Bob,...",0.784737
3,1,Alice,1,0,1,2,BasketballCourt,1,0,1,...,0,0,0,0,0,0,0,0,"play_basketball(Alice, Bob, BasketballCourt)\r\n",0.686699
4,1,Alice,1,0,1,2,Downtown,1,0,1,...,0,0,0,0,0,0,0,0,"travel(Alice, Downtown, BasketballCourt) steal...",0.698028


In [7]:
df.results.value_counts()

travel(Bob, HomeB, BasketballCourt) steal(Bob, Alice, Basketball, BasketballCourt) play_basketball(Bob, Alice, BasketballCourt)\r\n                                                        206
travel(Charlie, Downtown, BasketballCourt) kill(Charlie, Alice, Basketball, BasketballCourt)\r\n                                                                                           202
kill(Charlie, Alice, Basketball, HomeB)\r\n                                                                                                                                                199
travel(Alice, HomeB, BasketballCourt) play_basketball(Alice, Bob, BasketballCourt)\r\n                                                                                                     197
travel(Bob, Downtown, BasketballCourt) steal(Bob, Alice, Basketball, BasketballCourt) play_basketball(Bob, Alice, BasketballCourt)\r\n                                                     195
                                             

In [8]:

action = df.at[0,'results'].split(' ')[0]
action = action.replace('(',' ')
action = action.split(' ')
print(action)
only_action = action[0]
variables = action[1][:-1].split(', ')

print(only_action)
print(variables)

['travel', 'Alice,']
travel
['Alice']


In [9]:
df_effects = pd.read_csv('basketball_effects_nn.csv')

In [10]:
def removern(row):
    if 'results' in row.index:
        row.results = row.results.replace('\r','')
        row.results = row.results.replace('\n','')
        row.results = row.results.replace('\t','')
    if 'effect' in row.index:
        row.effect = row.effect.replace('\r','')
        row.effect = row.effect.replace('\n','')
        row.effect = row.effect.replace('\t','')
    return row

In [11]:
df = df.apply(removern,axis=1)

In [12]:
df_effects = df_effects.apply(removern,axis=1)

In [13]:
df_effects

,action,effect,effect_function
0,lose_support_authority,"!supports(char2,char3);support(char3) - 1",arg2_supports_arg3:=_0;arg3_support:-_1\n
1,travel,at(character) = to;,arg1_at:=_arg3
2,arrest,underArrest(character) = 1;,arg2_underArrest:=_1
3,steal,has(item) = thief ;angry(victim) = 1 ;clue(The...,arg3_has:=_arg1;arg2_angry:=_1;Theft_arg3_arg4...
4,play_basketball,angry(player1) = 0 ;angry(player2) = 0;,arg1_angry:=_0;arg2_angry:=_0
5,kill,"!alive(victim);clue(Murder, item, place);",arg2_alive:=_0;Murder_arg3_arg4_clues:=_1
6,find_clues,searched(place) = 1 ;,arg4_searched:=_1
7,share_clues,"believes(police2, clue(crime, item, place));",arg3_arg4_arg5_clues:=_1
8,suspect_of_crime,"suspect(citizen, crime);",arg2_suspect:=_arg3


In [14]:
def replace_effect(effect,args):
	for count,arg in enumerate(args,0):			
		effect = effect.replace('arg'+str(count+1),arg)
	
	return effect
					
						


In [15]:
def create_result(df ,row):
    
    
    actions = row.results.split(') ')

    action = actions[0]
    action = action.split('(')
    only_action = action[0]
    
    variables = action[1].replace(')','').split(', ')
    #print(variables)
    new_row = row
    effects= df_effects[df_effects.action == action[0]].effect_function
        
    if len(effects) > 0:
        for effect in str(effects.values[0]).split(';'):
            new_row.results = replace_effect(effect,variables) 
            df.loc[len(df)] = new_row

        #df.drop(index)
    return only_action
        #musi być action_variables albo najlepiej co się zmienia
    

In [16]:
df2 = pd.DataFrame(columns = df.columns)
for index, row in df.iterrows():
    create_result(df2,row)
    
#df['result'] = df.apply(create_result,axis=1)
#df2 = df2.drop(['time','scenario'],axis=1)

df = df2
df.rename({'results':'result'},axis=1,inplace=True)


In [117]:
df = df2
df

,scenario,acting_char,Alice_alive,Alice_underArrest,Alice_angry,Alice_suspect,Alice_at,Bob_alive,Bob_underArrest,Bob_angry,...,Theft_Bat_BasketballCourt_clues,Theft_Bat_Downtown_clues,Murder_Basketball_HomeB_clues,Murder_Basketball_BasketballCourt_clues,Murder_Basketball_Downtown_clues,Murder_Bat_HomeB_clues,Murder_Bat_BasketballCourt_clues,Murder_Bat_Downtown_clues,result,time
0,1,Alice,1,0,1,2,HomeB,1,0,1,...,0,0,0,0,0,0,0,0,Alice_at:=_Downtown,0.708981
1,1,Charlie,1,0,1,2,BasketballCourt,1,0,0,...,0,0,0,0,0,0,0,0,Charlie_at:=_BasketballCourt,0.70107
2,1,Bob,1,0,1,2,BasketballCourt,1,0,0,...,0,0,0,0,0,0,0,0,Bob_at:=_BasketballCourt,0.784737
3,1,Alice,1,0,1,2,BasketballCourt,1,0,1,...,0,0,0,0,0,0,0,0,Alice_angry:=_0,0.686699
4,1,Alice,1,0,1,2,BasketballCourt,1,0,1,...,0,0,0,0,0,0,0,0,Bob_angry:=_0,0.686699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15378,4,Charlie,1,0,1,2,BasketballCourt,1,0,0,...,0,0,0,0,0,0,0,0,Charlie_at:=_BasketballCourt,2.454284
15379,4,Charlie,1,0,1,2,HomeB,1,0,0,...,0,0,0,0,0,0,0,0,Charlie_at:=_HomeB,2.213896
15380,4,Charlie,1,0,1,2,Downtown,1,0,1,...,0,0,0,0,0,0,0,0,Charlie_at:=_Downtown,2.4322
15381,4,Charlie,1,0,1,2,HomeB,1,0,0,...,0,0,0,0,0,0,0,0,Alice_alive:=_0,2.303277


In [32]:
categorical = [x for x in df.columns if df[x].dtype == 'string' and x != 'result'] 
df = df.convert_dtypes()
df = pd.get_dummies(df, columns = categorical)

In [20]:
# Extract text features
cats = df.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   df[col] = df[col].astype('category')

In [119]:
y = df2.result
X = df2.drop(['result','time','scenario'],axis=1)

In [121]:
# example of a ordinal encoding
from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder
# define data
feature_names = list(df2.columns)[:-2]

# define ordinal encoding
encoder = OrdinalEncoder()
# transform data
X = pd.DataFrame(encoder.fit_transform(asarray(X)))
#y = pd.DataFrame(encoder.fit_transform(asarray(y).reshape(-1, 1)))
y = pd.get_dummies(y)



In [110]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)

In [25]:
import xgboost as xgb

# Create regression matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [27]:
params = {"objective": "multi:softprob", "num_class": 32}
n = 100

results = xgb.cv(
   params, dtrain_clf,
   num_boost_round=n,
   nfold=5,
   metrics=["mlogloss", "auc", "merror"],
)

In [28]:
results['test-auc-mean'].max()

0.9771859625927448

In [129]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth=6, n_estimators=10)
model.fit(asarray(X_train), asarray(y_train))
# dump it to a text file
#model.get_booster().dump_model('xgb_model.txt', with_stats=True)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [130]:
y_train_pred = model.predict(X_train)
y_train_pred_score = model.predict_proba(X_train)[:, 1]

y_test_pred = model.predict(X_test)
y_test_pred_score = model.predict_proba(X_test)[:, 1]

In [131]:
accuracy = model.score(X_test, y_test)
print("Accuracy")
print(accuracy)

Accuracy
0.4777380565485863


In [143]:
action='mind_char1'
char = action.split('_')[1]
num = char.split('char')[1]
num

'1'

In [54]:
import te2rules
from te2rules.explainer import ModelExplainer

In [132]:
model_explainer = ModelExplainer(
    model=model, 
    feature_names=feature_names
)

rules = model_explainer.explain(
    X=X_train, y=y_train_pred,
    num_stages = 5,               # stages can be between 1 and max_depth 
    min_precision = 0.95,          # higher min_precision can result in rules with more terms overfit on training data 
    jaccard_threshold = 0.4        # lower jaccard_threshold speeds up the rule exploration, but can miss some good rules
)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [126]:
y

,Alice_alive:=_0,Alice_angry:=_0,Alice_angry:=_1,Alice_at:=_BasketballCourt,Alice_at:=_Downtown,Alice_at:=_HomeB,Basketball_has:=_Alice,Basketball_has:=_Bob,Basketball_has:=_Charlie,Bat_has:=_Charlie,...,Murder_Basketball_HomeB_clues:=_1,Murder_Bat_BasketballCourt_clues:=_1,Murder_Bat_Downtown_clues:=_1,Murder_Bat_HomeB_clues:=_1,Theft_Basketball_BasketballCourt_clues:=_1,Theft_Basketball_Downtown_clues:=_1,Theft_Basketball_HomeB_clues:=_1,Theft_Bat_BasketballCourt_clues:=_1,Theft_Bat_Downtown_clues:=_1,Theft_Bat_HomeB_clues:=_1
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15381,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
